In [1]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.optim import Adam
from torch.utils.data import DataLoader,random_split
from torchvision import transforms
from torchvision.datasets import ImageFolder

In [2]:
transform = transforms.Compose(
[transforms.Resize((50,50)),
 transforms.ToTensor(),
 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

def load_dataset_from_folder(batch_size,num_workers,shuffle=True):
    path = 'data/train/'
    all_data = ImageFolder(
        root = path,
        transform = transform
    )
    
    train_size = int(0.9 * len(all_data))
    validation_size = len(all_data) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(all_data, [train_size, validation_size])

    training_data_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=shuffle
    )
    
    validation_dataset_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=shuffle
    )
        
    return training_data_loader,validation_dataset_loader

class Net(nn.Module):
    fin_x = 0
    fin_y = 0
    image_x = 50 #36
    image_y = 50 #41
    k_size = 3
    stride = 2
    pool_size = 2
    def __init__(self):
        super(Net, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, self.k_size, self.stride),
            nn.ReLU(),
            nn.MaxPool2d(self.pool_size, self.pool_size))

        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, self.k_size, self.stride),
            nn.ReLU(),
            nn.MaxPool2d(self.pool_size, self.pool_size))
        
        self.drop_out = nn.Dropout()

        
        tmpx = int((int((self.image_x-self.k_size)/self.stride)+1)/self.pool_size)
        tmpy = int((int((self.image_y-self.k_size)/self.stride)+1)/self.pool_size)
        
        self.fin_x = int((int((tmpx-self.k_size)/self.stride)+1)/self.pool_size)
        self.fin_y = int((int((tmpy-self.k_size)/self.stride)+1)/self.pool_size)
        
        self.fc1 = nn.Sequential(
            nn.Linear(64 * self.fin_x * self.fin_y, 2048),
            nn.ReLU()
        )
        
        self.fc2 = nn.Sequential(
            nn.Linear(2048, 4),
            nn.ReLU()
        )
        
        self.sof = nn.LogSoftmax()

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(-1, 64 * self.fin_x * self.fin_y)
        x = self.drop_out(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.sof(x)
        return x

# net = Net()
# net = net.cuda()

In [3]:
model = Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.07)
# defining the loss function
criterion = nn.CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    print('Found Cuda')
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)
# criterion = nn.NLLLoss()
# optimizer = optim.Adam(net.parameters(), lr=0.00001)

Net(
  (layer1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (drop_out): Dropout(p=0.5, inplace=False)
  (fc1): Sequential(
    (0): Linear(in_features=256, out_features=2048, bias=True)
    (1): ReLU()
  )
  (fc2): Sequential(
    (0): Linear(in_features=2048, out_features=4, bias=True)
    (1): ReLU()
  )
  (sof): LogSoftmax()
)


In [4]:
batch_size = 100
shuffle = True
num_workers = 4

train_generator,test_generator = load_dataset_from_folder(batch_size,num_workers,shuffle)

# train_set = my_dataloader('test_x_0_100_rgb.npy', 'test_y_0_100_rgb.npy')
# train_generator = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4)

In [5]:
print(len(train_generator.dataset))
print(len(test_generator.dataset))

720
80


In [6]:
for epoch in range(10):
    running_loss = 0.0
    validation_loss = 0.0
    for step,(x,y) in enumerate(train_generator):
#         print(x.shape)
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print('[%d] Training loss: %.3f' %(epoch + 1, running_loss))
    with torch.no_grad():
        for step,(x,y) in enumerate(test_generator):
            outputs = model(x)
            loss = criterion(outputs, y)
            validation_loss += loss.item()
    print('[%d] Validation loss: %.3f' %(epoch + 1, validation_loss))
            

/home/ankurshaswat/MyFiles/Repos/COL780/env/lib/python3.7/site-packages/ipykernel_launcher.py:82: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


[1] Training loss: 97.293
[1] Validation loss: 1.386
[2] Training loss: 11.090
[2] Validation loss: 1.386
[3] Training loss: 11.090
[3] Validation loss: 1.386
[4] Training loss: 11.090
[4] Validation loss: 1.386


KeyboardInterrupt: 

In [7]:
# print(train_set[0])
for x,y in train_generator:
    print(x.shape)
    preds_Test = torch.argmax(model(x),1)
    test_examples = x.numpy()
    gold = y.numpy()
    predictions = preds_Test.numpy()
    break
    
#     print(preds_Test)

torch.Size([100, 3, 50, 50])


/home/ankurshaswat/MyFiles/Repos/COL780/env/lib/python3.7/site-packages/ipykernel_launcher.py:82: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [8]:
# test_result_np=test_result.numpy()
# print(preds_Test.numpy())
print("----------Test Accuracy-----------")
print(accuracy_score(gold, predictions))
print(gold,predictions)
print(confusion_matrix(gold, predictions))
print(classification_report(gold, predictions))

----------Test Accuracy-----------
0.3
[2 0 3 3 2 1 0 3 0 2 2 3 2 2 2 3 1 3 1 0 3 1 0 0 0 3 2 2 2 1 0 2 3 2 0 2 1
 3 1 2 3 1 3 0 0 3 2 3 2 2 0 3 1 3 1 0 2 3 0 3 3 0 2 3 0 3 3 2 1 3 2 2 2 3
 2 0 1 0 1 3 3 2 3 3 2 2 0 2 1 2 2 3 3 2 2 0 0 2 0 1] [3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
[[ 0  0  0 22]
 [ 0  0  0 15]
 [ 0  0  0 33]
 [ 0  0  0 30]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        22
           1       0.00      0.00      0.00        15
           2       0.00      0.00      0.00        33
           3       0.30      1.00      0.46        30

    accuracy                           0.30       100
   macro avg       0.07      0.25      0.12       100
weighted avg       0.09      0.30      0.14       100



/home/ankurshaswat/MyFiles/Repos/COL780/env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
# with torch.no_grad():
#     output = model(train_x)
    
# softmax = torch.exp(output).cpu()
# prob = list(softmax.numpy())
# predictions = np.argmax(prob, axis=1)

# # accuracy on training set
# accuracy_score(train_y, predictions)

In [ ]:
# # prediction for validation set
# with torch.no_grad():
#     output = model(val_x)

# softmax = torch.exp(output).cpu()
# prob = list(softmax.numpy())
# predictions = np.argmax(prob, axis=1)

# # accuracy on validation set
# accuracy_score(val_y, predictions)

In [ ]:
# # loading test images
# test_img = []
# for img_name in tqdm(test['id']):
#     # defining the image path
#     image_path = 'test_ScVgIM0/test/' + str(img_name) + '.png'
#     # reading the image
#     img = imread(image_path, as_gray=True)
#     # normalizing the pixel values
#     img /= 255.0
#     # converting the type of pixel to float 32
#     img = img.astype('float32')
#     # appending the image into the list
#     test_img.append(img)

# # converting the list to numpy array
# test_x = np.array(test_img)
# test_x.shape

In [ ]:
# # converting training images into torch format
# test_x = test_x.reshape(10000, 1, 28, 28)
# test_x  = torch.from_numpy(test_x)
# test_x.shape

In [ ]:
# # generating predictions for test set
# with torch.no_grad():
#     output = model(test_x)

# softmax = torch.exp(output).cpu()
# prob = list(softmax.numpy())
# predictions = np.argmax(prob, axis=1)